# Groene boekje part 2: wordform links

In [the first Groene Boekje notebook](groene_boekje.ipynb) we cleaned just for wordforms. Now we will extend that with wordform links based on the relations in the rows. For this we will probably also need to modify dbutils.

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload

In [ ]:
import ticclat.dbutils
import ticclat.ticclat_schema
import pandas as pd
import numpy as np
import tqdm

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy_utils import database_exists, create_database

In [ ]:
# Read information to connect to the database and put it in environment variables
import os
with open('ENVVARS.txt') as f:
    for line in f:
        parts = line.split('=')
        if len(parts) == 2:
            os.environ[parts[0]] = parts[1].strip()

In [ ]:
db_name = 'ticclat'
os.environ['dbname'] = db_name

In [ ]:
# engine = create_engine("mysql://{}:{}@localhost/{}".format(os.environ['user'], 
#                                                            os.environ['password'], 
#                                                            os.environ['dbname']))
# if not database_exists(engine.url):
#     create_database(engine.url)

# print(database_exists(engine.url))

# Session = sessionmaker(bind=engine)

In [ ]:
# from sqlalchemy import inspect

# inspector = inspect(engine)

In [ ]:
# # Get table information
# print(inspector.get_table_names())

# Load Groene Boekje data into Pandas

In [ ]:
GB_basepath = "/Users/pbos/projects/ticclat/data/GB/"

In [ ]:
GB1914_path = GB_basepath + "1914/22722-8.txt"
GB1995_path = GB_basepath + "1995-2005/1995/GB95_002.csv"
GB2005_path = GB_basepath + "1995-2005/2005/GB05_002.csv"

In [ ]:
# df_GB1995 = pd.read_csv(GB1995_path, sep=';', names=["word", "syllables", "see also", "disambiguation",
#                                                      "grammatical tag", "article",
#                                                      "plural/past/attrib", "plural/past/attrib syllables",
#                                                      "diminu/compara/past plural", "diminu/compara/past plural syllables",
#                                                      "past perfect/superla", "past perfect/superla syllables"],
#                         encoding='utf8') # encoding necessary for later loading into sqlalchemy!

In [ ]:
# # df_GB1995[~df_GB1995["see also"].isnull()].sample(10)
# # df_GB1995[~df_GB1995["disambiguation"].isnull()].sample(10)
# df_GB1995[~df_GB1995["disambiguation"].isnull() & df_GB1995["disambiguation"].str.contains(' ')].sample(10)

## Clean-up
Clean up will be different now that we also need links.
- At first we need to retain the columns while cleaning them, because we'll need the rows to define links.
- The cleaning of the wordform columns will be the same as before, but now per column instead of all in one go.
- However, because we must retain rows, things like splitting on commas will now mean the entire row must be duplicated.

Some notes picked up from random sample checking:
- The disambiguation words can also have a duplicate word numbers! These are irrelevant to us, since we only deal with word forms, not semantics, so we have to remove them there as well.

### Links
The different types of links we could extract from this table are:

- "see also" (column 3): this is usually an **spelling variant**, so very relevant in our case. Both are **correct** words.
- "disambiguation (column 4):
    + This column is always between parentheses.
    + Usually it is a semantically very similar word.
    + Sometimes the disambiguation is in multiple words. In this case (as opposed to the multiple words in wordform columns) multiple words are more often really separate words, like `slechte waar` (for `kamelot2`) or `het slopen` (for `sloop1`), which are really two separate wordforms, not a "multi-word wordform". We probably can't count on this, but we could do a lookup of the separate words. Even if we did that, this column entry is really almost more like a sentence, an explanation of the word, so the separate words cannot be entered into the database as wordforms, nor can they be entered separately, because their combination makes up their meaning, which it will lose when taken apart.
    + There are even entries that have multiple disambiguating words, separated by a comma. These can easily be used separately.
    + A lot of them have `(andere bett.)` or (more rarely) `(andere bet.)` which I guess means "other meaning(s)", not sure though.
    + So to sum up, what we can do with this column:
        * Remove `(andere bett.)`
        * Strip parentheses
        * Split by comma
        * **First approximation**: remove multi-word entries
        * *Use remaining words as **semantic** links*
        * Again, both are **correct** words.
- Columns 7, 9 and 11 give us **morphological links** of different types. Again, all **correct** words.
    + We could in principle deduce the type from the grammatical tag of the word in column 5, but will no do so for now.

# Clean up links

This time we clean up the dataframe as a table, not as a single row of wordforms.

We can drop a few columns though.

In [ ]:
# link_data = df_GB1995.drop(["syllables", "grammatical tag", "article",
#                             "plural/past/attrib syllables", "diminu/compara/past plural syllables", "past perfect/superla syllables"], axis=1)

In [ ]:
# link_data.sample(10)

## Clean columns 2 and 3

We have to remove `zie ook ` (see also ) from column 2.

For column 3 we will:
- strip the parentheses
- remove multi-word entries (including especially `andere bett.`).

### Clean "see also" (column 2)

In [ ]:
# link_data['see also'] = link_data['see also'].str.replace('zie ook ', '')

In [ ]:
# link_data[~link_data['see also'].isnull()].sample(5)

### Clean "disambiguation" (column 3)

In [ ]:
# link_data['disambiguation'] = link_data['disambiguation'].str.strip('()')

In [ ]:
# link_data['disambiguation'][link_data['disambiguation'] == 'andere bett.'] = None
# link_data['disambiguation'][link_data['disambiguation'] == 'andere bet.'] = None

In [ ]:
# link_data[~link_data['disambiguation'].isnull()].sample(5)

1. First remove the isolated multi-words, i.e. the ones without commas.
2. Then, if necessary (turns out, it's only 20-30 rows left), let's make a nice regex to replace multi-words, both isolated ones and ones in comma separated lists.

In [ ]:
link_data['disambiguation'][link_data['disambiguation'].str.contains(" ", na=False)
                            & ~link_data['disambiguation'].str.contains(",", na=False)] = None

In [ ]:
link_data['disambiguation'][link_data['disambiguation'].str.contains("[^,] ", na=False)] = (
    link_data['disambiguation']
     [link_data['disambiguation'].str.contains("[^,] ", na=False)]
     .str.split(', ')
     .map(lambda x: [i for i in x if not ' ' in i])
     .map(lambda x: None if len(x) == 0 else ', '.join(x))
)

In [ ]:
link_data['disambiguation'][link_data['disambiguation'].str.contains("[^,] ", na=False)]

In [ ]:
link_data['disambiguation'][link_data['disambiguation'].str.contains(" ", na=False)].sample(5)

One remaining pesky thing: `kippenloop, -korf`. Is this a pattern?

In [ ]:
link_data['disambiguation'][link_data['disambiguation'].str.contains(", -", na=False)]

Nope, so let's just get rid of it here and now.

In [ ]:
link_data['disambiguation'][link_data['disambiguation'].str.contains(", -", na=False)] = None

In [ ]:
link_data.sample(10)

## Remove empty lines

For links, we don't need the rows that only have a `word` entry, since these by definition have no links in that row. They may be linked to another word through the `see also` column, but then the word will also be in the row of the word where it is in the `see also` column, so the word's row itself can safely be removed.

In [ ]:
link_data = link_data.dropna(how='all', subset=["see also", "disambiguation", "plural/past/attrib", "diminu/compara/past plural", "past perfect/superla"])

# Convert to links!

Actually, for now, we can keep this rather simple: we just make a table with two columns, where the first column has the wordforms in the `word` column and the second has the wordforms in the other columns.

We will then split comma separated words in a second pass to keep things simple.

In [ ]:
link_df = (link_data.set_index('word').stack().reset_index().drop('level_1', axis=1)
                    .rename({'word': 'wordform_1', 0: 'wordform_2'}, axis=1))

In [ ]:
link_df.head()

In [ ]:
has_comma1 = link_df['wordform_1'].str.contains(',')
link_df = pd.concat((link_df[~has_comma1],) + tuple(pd.DataFrame({'wordform_1': row['wordform_1'].split(', '),
                                                                  'wordform_2': (row['wordform_2'],) * len(row['wordform_1'].split(', '))})
                                                     for ix, row in link_df[has_comma1].iterrows()))

has_comma2 = link_df['wordform_2'].str.contains(',')
link_df = pd.concat((link_df[~has_comma2],) + tuple(pd.DataFrame({'wordform_1': (row['wordform_1'],) * len(row['wordform_2'].split(', ')),
                                                                  'wordform_2': row['wordform_2'].split(', ')})
                                                     for ix, row in link_df[has_comma2].iterrows()))

In [ ]:
link_df.tail(12)

# Clean wordforms

Almost like in [the first Groene Boekje notebook](groene_boekje.ipynb), but now per column. The main changes are:
- We do it per column
- We must not do `.unique()` at the end, because duplicate words in a column may be linked to different words in the other column! Unique should be row-based.

We also redo the above in one go in the next cell.

In [ ]:
link_data = df_GB1995.drop(["syllables", "grammatical tag", "article",
                            "plural/past/attrib syllables", "diminu/compara/past plural syllables", "past perfect/superla syllables"], axis=1)

# clean link_data
link_data['see also'] = link_data['see also'].str.replace('zie ook ', '')
link_data['disambiguation'] = link_data['disambiguation'].str.strip('()')
link_data['disambiguation'][link_data['disambiguation'] == 'andere bett.'] = None
link_data['disambiguation'][link_data['disambiguation'] == 'andere bet.'] = None
link_data['disambiguation'][link_data['disambiguation'].str.contains(" ", na=False)
                            & ~link_data['disambiguation'].str.contains(",", na=False)] = None
link_data['disambiguation'][link_data['disambiguation'].str.contains("[^,] ", na=False)] = (
    link_data['disambiguation']
     [link_data['disambiguation'].str.contains("[^,] ", na=False)]
     .str.split(', ')
     .map(lambda x: [i for i in x if not ' ' in i])
     .map(lambda x: None if len(x) == 0 else ', '.join(x))
)
link_data['disambiguation'][link_data['disambiguation'].str.contains(", -", na=False)] = None
link_data = link_data.dropna(how='all', subset=["see also", "disambiguation", "plural/past/attrib", "diminu/compara/past plural", "past perfect/superla"])

# convert to link_df
link_df = (link_data.set_index('word').stack().reset_index().drop('level_1', axis=1)
                    .rename({'word': 'wordform_1', 0: 'wordform_2'}, axis=1))
has_comma1 = link_df['wordform_1'].str.contains(',')
link_df = pd.concat((link_df[~has_comma1],) + tuple(pd.DataFrame({'wordform_1': row['wordform_1'].split(', '),
                                                                  'wordform_2': (row['wordform_2'],) * len(row['wordform_1'].split(', '))})
                                                     for ix, row in link_df[has_comma1].iterrows()))

has_comma2 = link_df['wordform_2'].str.contains(',')
link_df = pd.concat((link_df[~has_comma2],) + tuple(pd.DataFrame({'wordform_1': (row['wordform_1'],) * len(row['wordform_2'].split(', ')),
                                                                  'wordform_2': row['wordform_2'].split(', ')})
                                                     for ix, row in link_df[has_comma2].iterrows()))

link_df = link_df.reset_index(drop=True)

In [ ]:
def clean_wordform_series(wordform_series, remove_duplicates=False):
    # remove colons
    wordform_series = wordform_series.str.replace(':', '')
    # strip whitespace
    wordform_series = wordform_series.str.strip()
    # remove duplicate word footnote numbers
    duplicates = wordform_series.str.contains('[0-9]$', regex=True)
    wordform_series = pd.concat((wordform_series[~duplicates], wordform_series[duplicates].str.replace('[0-9]$', '', regex=True)))
    # remove parentheses around some words
    wordform_series = wordform_series.sort_values().str.strip("()")
    # remove abbreviations
    abbreviation = wordform_series.str.contains('\.$')
    wordform_series = wordform_series[~abbreviation]
    
    if remove_duplicates:
        wordform_series = pd.Series(wordform_series.unique())
    return wordform_series

In [ ]:
link_clean_df = link_df.copy()
link_clean_df['wordform_1'] = clean_wordform_series(link_clean_df['wordform_1'])
link_clean_df['wordform_2'] = clean_wordform_series(link_clean_df['wordform_2'])
# some links will be removed (abbreviations), so drop those rows
link_clean_df = link_clean_df.dropna()

In [ ]:
len(link_df) - len(link_clean_df)

In [ ]:
def check_cleanliness_wordform_series(wordform_series, head=5):
    print("Random sample:")
    display(wordform_series.sample(10))
    print("Colons, periods:")
    display(wordform_series[wordform_series.str.contains(':')].head(head))
    display(wordform_series[wordform_series.str.contains('.', regex=False)].head(head))
    print("White space padding:")
    display(wordform_series[wordform_series.str.contains('^ | $')].head(head))
    print("Trailing numbers:")
    display(wordform_series[wordform_series.str.contains('[0-9]$', regex=True)].head(head))
    print("Parentheses:")
    display(wordform_series[wordform_series.str.contains('\(|\)', regex=True)].head(head))
    print("Abbreviations:")
    display(wordform_series[wordform_series.str.contains('\.$')].head(head))
    
    print("Finally, just the first entries of sorted df:")
    display(wordform_series.sort_values().head(head))
    display(wordform_series.sort_values().tail(head))

In [ ]:
check_cleanliness_wordform_series(link_df['wordform_1'])
check_cleanliness_wordform_series(link_df['wordform_2'])

In [ ]:
check_cleanliness_wordform_series(link_clean_df['wordform_1'])
check_cleanliness_wordform_series(link_clean_df['wordform_2'])

### Normalizing diacritics

In [ ]:
# note that these are regex formatted, i.e. with special characters escaped
diacritic_markers = {'@`': '\u0300',    # accent grave
                     "@\\'": '\u0301',  # accent aigu
                     '@\\\\': '\u0308', # trema
                     '@\+': '\u0327',   # cedilla
                     '@\^': '\u0302',   # accent circumflex
                     '@=': '\u0303',    # tilde
                     '@@': "'",         # apostrophe (not actually a diacritic)
                     '@2': '\u2082',    # subscript 2
                     '@n': '\u0308n'    # trema followed by n
                    }

In [ ]:
for column in link_clean_df:
    for marker, umarker in diacritic_markers.items():
        link_clean_df[column] = link_clean_df[column].str.replace(marker, umarker)

In [ ]:
link_clean_df.sort_values(by=link_clean_df.columns.tolist()).sample(10)

# Load Groene Boekje LINKS DataFrames into TICCLAT database

Here we use the example of the [Twente spelling correction notebook](twente_spelling_correction.ipynb).

In [ ]:
# %time

# with ticclat.dbutils.session_scope(Session) as session:
#     lexicon = session.query(ticclat.ticclat_schema.Lexicon).filter(ticclat.ticclat_schema.Lexicon.lexicon_name=='Groene Boekje 1995').first()
#     if lexicon is None:
#         raise Exception("No lexicon found!")
#     for idx, row in tqdm.tqdm(link_clean_df.iterrows(), total=link_clean_df.shape[0]):
#         wf = session.query(ticclat.ticclat_schema.Wordform).filter(ticclat.ticclat_schema.Wordform.wordform == row['wordform_1'].encode('utf8')).first()
#         corr = session.query(ticclat.ticclat_schema.Wordform).filter(ticclat.ticclat_schema.Wordform.wordform == row['wordform_2'].encode('utf8')).first()        
#         wf.link_with_metadata(corr, True, True, lexicon)

Hmm, apparently there is a None / NaN somewhere in the table...

In [ ]:
any(link_clean_df.isna())

In [ ]:
any(link_clean_df.index.isna()), any(link_clean_df['wordform_1'].isna()), any(link_clean_df['wordform_2'].isna())

But where?

Maybe the problem is with the original ingestion, there could be a wordform in this links DataFrame that was filtered out of the original wordforms DataFrame...

In [ ]:
link_clean_df.iloc[4559-1]

Hm, yeah that quote might be problematic...

## After fixing reading in of the csv file

Again, all in one, with fixed original df.

In [ ]:
import ticclat.ingest.groene_boekje

In [ ]:
link_df = ticclat.ingest.groene_boekje.create_GB95_link_df(ticclat.ingest.groene_boekje.load_GB95(GB1995_path))

In [ ]:
# # %time

# with ticclat.dbutils.session_scope(Session) as session:
#     lexicon = session.query(ticclat.ticclat_schema.Lexicon).filter(ticclat.ticclat_schema.Lexicon.lexicon_name=='Groene Boekje 1995').first()
#     if lexicon is None:
#         raise Exception("No lexicon found!")
#     for idx, row in tqdm.tqdm(link_df.iterrows(), total=link_df.shape[0]):
#         wf = session.query(ticclat.ticclat_schema.Wordform).filter(ticclat.ticclat_schema.Wordform.wordform == row['wordform_1'].encode('utf8')).first()
#         corr = session.query(ticclat.ticclat_schema.Wordform).filter(ticclat.ticclat_schema.Wordform.wordform == row['wordform_2'].encode('utf8')).first()        
#         wf.link_with_metadata(corr, True, True, lexicon)

Fuck, still the same crap it seems...

In [ ]:
# link_df.loc[4556]

Ah, the problem turned out to be the `engine` URL, it was missing the `?charset=utf8mb4` part! Now we can also leave off the `.encode('utf8')` crap.

Ok, again:

In [ ]:
engine8 = create_engine("mysql://{}:{}@localhost/{}?charset=utf8mb4".format(os.environ['user'], 
                                                                            os.environ['password'], 
                                                                            os.environ['dbname']))
Session8 = sessionmaker(bind=engine8)

In [ ]:
# # %time

# with ticclat.dbutils.session_scope(Session8) as session:
#     lexicon = session.query(ticclat.ticclat_schema.Lexicon).filter(ticclat.ticclat_schema.Lexicon.lexicon_name=='Groene Boekje 1995').first()
#     if lexicon is None:
#         raise Exception("No lexicon found!")
#     for idx, row in tqdm.tqdm(link_df.iterrows(), total=link_df.shape[0]):
#         wf = session.query(ticclat.ticclat_schema.Wordform).filter(ticclat.ticclat_schema.Wordform.wordform == row['wordform_1']).first()
#         corr = session.query(ticclat.ticclat_schema.Wordform).filter(ticclat.ticclat_schema.Wordform.wordform == row['wordform_2']).first()
#         if corr is None:
#             print(row['wordform_2'])
#         wf.link_with_metadata(corr, True, True, lexicon)

... man, what now?

In [ ]:
with ticclat.dbutils.session_scope(Session8) as session:
    lexicon = session.query(ticclat.ticclat_schema.Lexicon).filter(ticclat.ticclat_schema.Lexicon.lexicon_name=='Groene Boekje 1995').first()
    row = link_df.loc[4554]
    wf = session.query(ticclat.ticclat_schema.Wordform).filter(ticclat.ticclat_schema.Wordform.wordform == row['wordform_1']).first()
    corr = session.query(ticclat.ticclat_schema.Wordform).filter(ticclat.ticclat_schema.Wordform.wordform == row['wordform_2']).first()
    print(wf)
    print(corr)

New notebook to figure this out: [GB_cleaning_problems](../GB_cleaning_problems.ipynb)

Ok, figured it out, it was a problem with the wordforms. So now we run again and everything will be fine:

In [ ]:
# %time

with ticclat.dbutils.session_scope(Session8) as session:
    lexicon = session.query(ticclat.ticclat_schema.Lexicon).filter(ticclat.ticclat_schema.Lexicon.lexicon_name=='Groene Boekje 1995').first()
    if lexicon is None:
        raise Exception("No lexicon found!")
    for idx, row in tqdm.tqdm(link_df.iterrows(), total=link_df.shape[0]):
        wf = session.query(ticclat.ticclat_schema.Wordform).filter(ticclat.ticclat_schema.Wordform.wordform == row['wordform_1']).first()
        corr = session.query(ticclat.ticclat_schema.Wordform).filter(ticclat.ticclat_schema.Wordform.wordform == row['wordform_2']).first()
        if corr is None:
            print(row['wordform_2'])
        wf.link_with_metadata(corr, True, True, lexicon)

Put this in the script.